In [1]:
%%html
<style>
table {float:left}
</style>

# Fix the table alignment in the next cell

# POC of reading/writing a feather dataframe

There is a nice writeup of file format comparisons [here](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d), this notebook is just for my own exploration.

With
* 1M Rows
* 10 Categorical Columns
* 10 Numeric Columns

With Test 1 using `object` for the categorical columns

| File Type | Write Speed | Read Speed | File Size |
|-----------|-------------|------------|-----------|
| Feather   | 954 ms      | 489 ms     | 443M      |
| CSV/GZ    | 34 s        | 8.47 s     | 143M      |

<br/><br/> <br/><br/>
With Test 2 using `pd.Categorical` for the categorical columns.  This is there Feather really shins as the categorical variables are heavily leveraged for reading/writing and file size.  Also when the dataframe is read in the categorical dtypes are automatically pulled from the file.  The csv file has object column types.
<br/><br/>

| File Type | Write Speed | Read Speed | File Size |
|-----------|-------------|------------|-----------|
| Feather   | 60.9 ms     | 61.8 ms    | 88M       |
| CSV/GZ    | 32.8 s      | 7.99 s     | 143M      |

<br/><br/> <br/><br/>

(Note: These numbers may not 100% match below here, they are from a single run)

In [2]:
%%capture
!conda install --yes -c conda-forge feather-format
!pip install hurry.filesize

In [3]:
# Number of numerical columns to be generated in the test dataframe
numerical_col_count = 10

# Number of categorical columns to be generated in the test dataframe
categorical_col_count = 10

# Number of rows for the test dataframe
n_rows = 1000000

In [4]:
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal
import feather # Don't need this, just double checking that it's installed
import tempfile
import os
from hurry.filesize import size

In [5]:
# copied from https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

def generate_dataset(n_rows, num_count, cat_count, max_nan=0.1, max_cat_size=100):
    """Randomly generate datasets with numerical and categorical features.
    
    The numerical features are taken from the normal distribution X ~ N(0, 1).
    The categorical features are generated as random uuid4 strings with 
    cardinality C where 2 <= C <= max_cat_size.
    
    Also, a max_nan proportion of both numerical and categorical features is replaces
    with NaN values.
    """
    dataset, types = {}, {}
    
    def generate_categories():
        from uuid import uuid4
        category_size = np.random.randint(2, max_cat_size)
        return [str(uuid4()) for _ in range(category_size)]
    
    for col in range(num_count):
        name = f'n{col}'
        values = np.random.normal(0, 1, n_rows)
        nan_cnt = np.random.randint(1, int(max_nan*n_rows))
        index = np.random.choice(n_rows, nan_cnt, replace=False)
        values[index] = np.nan
        dataset[name] = values
        types[name] = 'float32'
        
    for col in range(cat_count):
        name = f'c{col}'
        cats = generate_categories()
        values = np.array(np.random.choice(cats, n_rows, replace=True), dtype=object)
        nan_cnt = np.random.randint(1, int(max_nan*n_rows))
        index = np.random.choice(n_rows, nan_cnt, replace=False)
        values[index] = np.nan
        dataset[name] = values
        types[name] = 'object'
    
    return pd.DataFrame(dataset), types

In [6]:
df, types = generate_dataset(n_rows, numerical_col_count, categorical_col_count)
display(df.head(3))
display(types)

,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,-0.826762,0.255975,0.687763,-0.641403,0.203848,0.114130,-0.718027,-0.095465,1.049963,-0.015310,6eff2a18-8f5e-45cd-95fa-aeca47af178a,bc00f60e-b8ff-4291-8e58-afce86c43cbe,7e873652-e519-4d6a-98aa-83952f9b53f1,6c6c2571-43c8-463e-a083-284025a0c798,7f43943f-b6a5-4d3f-b6e0-1009726be4ca,72523ecb-cac3-4d25-829e-c11952534a04,bf956841-ce1a-49fe-9c8b-c08ff17e6055,c1d98ede-d098-4c6d-9365-f9f9d4748b4e,bea7e07a-233e-4a80-9134-aa053c0f4d8b,b742fdb9-ab3f-4d5f-9657-88bf5ce78eb8
1,-0.906168,-0.765864,-2.224771,1.096117,-1.219615,-1.394968,-0.299018,-0.314434,1.248955,2.858609,c785330b-7a89-4900-9536-539dcc3e1bb2,ee5ff891-ce6a-4779-be4c-ccc05e9bdfd2,7e873652-e519-4d6a-98aa-83952f9b53f1,5b506d05-75c4-4d66-9a70-aad028355123,b9de2301-529a-41d0-a268-ee34d4d575ac,f709fc88-b3a8-47ca-a75f-39831f8350c2,ae7107a6-2184-4d54-8f3c-3df4f3e5389c,NaN,3669ab7d-7c75-47be-b476-2e75dc3561dd,ad4e2c7b-936c-4c58-9aaa-a5587dfadeb6
2,0.193081,1.543829,0.652607,-0.383484,-2.034771,0.774398,-1.316463,0.486021,-1.305522,-0.755622,39a51649-c4d0-40bb-8437-f835325c1e0a,e1a3a4fb-c98b-43bc-9f4f-e27a18522896,e950899c-3c40-45ed-9dca-dcf5ec2e6e67,a851ea74-0d9b-4a0d-9f2f-fa804946c5df,9aee1c92-d2c1-4fb3-8bac-e22d5b3b1daa,9bae9c3b-6f4f-4b63-82f4-61048fe8626a,7174e1fe-0f5e-4f1e-a554-13edf15f4347,75f1f1ec-d488-4b76-b565-b98779e9ce3c,c1b4a6be-1147-49d4-8e30-eb3598561dab,bf2fb51e-8445-45d6-abc8-e15b4d4f6069


{'n0': 'float32',
 'n1': 'float32',
 'n2': 'float32',
 'n3': 'float32',
 'n4': 'float32',
 'n5': 'float32',
 'n6': 'float32',
 'n7': 'float32',
 'n8': 'float32',
 'n9': 'float32',
 'c0': 'object',
 'c1': 'object',
 'c2': 'object',
 'c3': 'object',
 'c4': 'object',
 'c5': 'object',
 'c6': 'object',
 'c7': 'object',
 'c8': 'object',
 'c9': 'object'}

# Speeds for reading/writing
* feather
* gzip/csv

In [7]:
out_feather_tmp_file = tempfile.NamedTemporaryFile(suffix='.feather')
out_gzip_csv_tmp_file = tempfile.NamedTemporaryFile(suffix='.csv.gz')

## Speed to write a feather file

In [8]:
%%time
df.to_feather(out_feather_tmp_file.name)

CPU times: user 683 ms, sys: 352 ms, total: 1.04 s
Wall time: 1.01 s


## Speed to read a feather file

In [9]:
%%time
feather_df = pd.read_feather(out_feather_tmp_file.name)

CPU times: user 433 ms, sys: 285 ms, total: 718 ms
Wall time: 455 ms


In [10]:
assert_frame_equal(df, feather_df)

## Speed to write a gziped csv

In [11]:
%%time
df.to_csv(out_gzip_csv_tmp_file.name, compression='gzip')

CPU times: user 35.3 s, sys: 147 ms, total: 35.5 s
Wall time: 35.5 s


## Speed to read a gziped csv

In [12]:
%%time
csv_gzip_df = pd.read_csv(out_gzip_csv_tmp_file.name)

CPU times: user 9.62 s, sys: 390 ms, total: 10 s
Wall time: 8.5 s


In [13]:
try:
    assert_frame_equal(df, csv_gzip_df)
    print("SUCCESS: Frames are equal!")
except:
    print("FAILURE: Frames are unequal!")

# By default the csv file has an index read in as an additional column
assert_frame_equal(df, csv_gzip_df.drop(columns=['Unnamed: 0']))

FAILURE: Frames are unequal!


# Comparison of File Sizes

In [14]:
feather_size_mb = size(os.path.getsize(out_feather_tmp_file.name))
csv_gzip_size_mb = size(os.path.getsize(out_gzip_csv_tmp_file.name))

print(f"Size of Feather File in MB: {feather_size_mb}")
print(f"Size of csv/gzip File in MB: {csv_gzip_size_mb}")

Size of Feather File in MB: 444M
Size of csv/gzip File in MB: 174M


# Use pd.Categorical

One benefit of Feather is that it stores the type of the object.

So let's try that with a categorical column

In [15]:
for col in df.dtypes[df.dtypes == 'object'].index:
    df[col] = df[col].astype('category')

In [16]:
%%time
df.to_feather(out_feather_tmp_file.name)

CPU times: user 136 ms, sys: 47.3 ms, total: 184 ms
Wall time: 62.8 ms


## Speed to read a feather file

In [17]:
%%time
feather_df = pd.read_feather(out_feather_tmp_file.name)

CPU times: user 158 ms, sys: 198 ms, total: 356 ms
Wall time: 62.4 ms


In [18]:
assert_frame_equal(df, feather_df)

## Speed to write a gziped csv

In [19]:
%%time
df.to_csv(out_gzip_csv_tmp_file.name, compression='gzip')

CPU times: user 34.8 s, sys: 176 ms, total: 35 s
Wall time: 35 s


## Speed to read a gziped csv

In [20]:
%%time
csv_gzip_df = pd.read_csv(out_gzip_csv_tmp_file.name)

CPU times: user 9.58 s, sys: 389 ms, total: 9.97 s
Wall time: 8.41 s


In [21]:
try:
    # By default the csv file has an index read in as an additional column
    assert_frame_equal(df, csv_gzip_df.drop(columns=['Unnamed: 0']))
    print("SUCCESS: Frames are equal!")
except:
    print("FAILURE: Frames are unequal!")
    
print("Starting to see the benefits of Feather here as the dtypes differ")
display(pd.concat([df.dtypes, csv_gzip_df.dtypes], axis=1, sort=True)
        .rename(columns={0:'dtypes_from_original_df', 1:'dtypes_from_csv_gzip_df'}))

FAILURE: Frames are unequal!
Starting to see the benefits of Feather here as the dtypes differ


,dtypes_from_original_df,dtypes_from_csv_gzip_df
Unnamed: 0,NaN,int64
c0,category,object
c1,category,object
c2,category,object
c3,category,object
c4,category,object
c5,category,object
c6,category,object
c7,category,object
c8,category,object


# Comparison of File Sizes

In [22]:
feather_size_mb = size(os.path.getsize(out_feather_tmp_file.name))
csv_gzip_size_mb = size(os.path.getsize(out_gzip_csv_tmp_file.name))

print(f"Size of Feather File in MB: {feather_size_mb}")
print(f"Size of csv/gzip File in MB: {csv_gzip_size_mb}")

Size of Feather File in MB: 88M
Size of csv/gzip File in MB: 174M
